In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
all_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")
area_code_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_AreaCodes.csv')

code_to_name_dict = area_code_df[['Area Code', 'Area']].set_index('Area Code').to_dict()['Area']
all_df.loc[:, 'Area'] = all_df['Area Code'].replace(code_to_name_dict)
lat_lon = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')

In [2]:
all_df = all_df.merge(
    lat_lon[['latitude', 'longitude', 'country']],
    how='left',
    left_on='Area',
    right_on='country',
)

In [3]:
def make_year_cols(start, end):
    return [f'Y{y}' for y in range(start, end)]

In [13]:
all_df[all_df['Item'].str.endswith('Total')]['Item'].unique()

array(['Cereals, Total', 'Citrus Fruit, Total', 'Meat, Total',
       'Milk, Total', 'Pulses, Total', 'Roots and Tubers, Total',
       'Treenuts, Total'], dtype=object)

# check country name

In [24]:
with open('./data/ne_110m.json', 'r') as f:
    text = f.read()

In [25]:
for country_name in all_df['Area'].unique():
    if text.find('"NAME":"{}"'.format(country_name)) == -1:
        print(country_name)


Antigua and Barbuda
Bahrain
Barbados
Belgium-Luxembourg
Cabo Verde
China; (including Taiwan)
China; Hong Kong SAR
China; Macao SAR
Comoros
Cook Islands
Czechoslovakia
Dominica
Eswatini
Ethiopia PDR
Faroe Islands
French Guyana
French Polynesia
Grenada
Guadeloupe
Kiribati
Maldives
Malta
Marshall Islands
Martinique
Mauritius
Micronesia (Federated States of)
Nauru
Niue
R�union
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
Sao Tome and Principe
Serbia and Montenegro
Seychelles
Singapore
South Sudan
Sudan (former)
Tokelau
Tonga
Tuvalu
USSR
Yugoslav SFR
World
Africa
Eastern Africa
Middle Africa
Northern Africa
Southern Africa
Western Africa
Americas
Northern America
Central America
Caribbean
South America
Asia
Central Asia
Eastern Asia
Southern Asia
South-eastern Asia
Western Asia
Europe
Eastern Europe
Northern Europe
Southern Europe
Western Europe
Oceania
Australia and New Zealand
Melanesia
Micronesia
Polynesia
European Union (27)
Least Developed Countries
Land Loc

# wrangle file

Map of productiono of cereal

In [15]:
df = all_df[
    (all_df['Item'].str.endswith('Total')) &\
    (all_df['Area Code'] <= 300) &\
    (all_df['Element'].isin(['Yield']))
][
    ['Area', "longitude", "latitude", 'Item', 'Element', 'Unit']
  + make_year_cols(2000, 2021)
].dropna(axis=0, how='any', subset=['latitude', 'longitude'])\
  .melt(
    id_vars=['Area', "longitude", "latitude", 'Item', 'Element', 'Unit'], 
    value_vars=make_year_cols(2000, 2021), 
    value_name='Quantity', 
    var_name='Year'
)

df['Year'] = df['Year'].str.strip('Y')
df.to_csv('data/processed/yield_major_categories_per_country_2000-2020.csv')

In [23]:
df[df['Element'] == 'Yield']['Quantity'].describe()

count      3612.000000
mean      33372.773533
std       29635.836714
min        1526.000000
25%       15032.000000
50%       27397.500000
75%       43079.250000
max      367619.000000
Name: Quantity, dtype: float64